In [3]:
from core.libs import pd
from core.db import get_engine
from core.schema_helpers import rename_columns_using_schema
from CruisesProcessor.general_importer import prepare_df_for_sql, save_inventory_to_sql, ensure_table
from CruisesProcessor.dead_alive_calculator import calculate_dead_alive
from CruisesProcessor.tree_id import split_by_id_validity
from core.doyle_calculator import calculate_doyle

# archivos = [
#     r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\Costa Rica\2021_ForestInventory\7-Database\Final\CostaRica_Database_All v1.1.xlsx",
#     r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\Mexico\2021_ForestInventory\6-Database\2021_Mexico_Database v1.0.xlsx"
#     ,
#     r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\Guatemala\2021_ForestInventory\6-Database\Database\2021_Guatemala_Database v1.2.xlsx",
#     r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\USA\2021_ForestInventory\9-Database\2021_US_Database v1.1.xlsx"
#     ]
# tablas_destino = ['inventory_cr_2021','inventory_mx_2021','inventory_gt_2021', 'inventory_us_2021'  ]

archivos = [
    r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\USA\2021_ForestInventory\9-Database\2021_US_Database v1.1.xlsx"
    ]
tablas_destino = ['inventory_us_2021'  ]
sheet = 'TreeList_Data'

engine = get_engine()

💻 Conectado a la base de datos helloworldtree


In [4]:
for archivo, tabla_destino in zip(archivos, tablas_destino):
    print(f"🌲 Procesando {archivo} → {tabla_destino}")
    df = pd.read_excel(archivo,sheet_name=sheet)
    df = df.drop(columns=["DAP (cm)", "THt (m)", "MHt (m)"], errors="ignore")
    df = rename_columns_using_schema(df)

    df = calculate_dead_alive(df, engine)
    df = calculate_doyle(df)

    df_ok, df_bad = split_by_id_validity(df)
    df_sql, _ = prepare_df_for_sql(df_ok)
    df_sql = df_sql.where(pd.notna(df_sql), None)
    ensure_table(df_sql, engine, tabla_destino)
    save_inventory_to_sql(df_sql, engine, tabla_destino)


    print("Ahí va :D")
    #print(f"  ✅ Guardado completado: {tabla_destino} ({len(df_sql)} filas)\n")


🌲 Procesando C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\USA\2021_ForestInventory\9-Database\2021_US_Database v1.1.xlsx → inventory_us_2021
🌳 Doyle recalculado
🔑 PRIMARY KEY agregada en 'id'

=== INICIO DE IMPORTACIÓN ===
💻 Conectado a la base de datos helloworldtree
✅ Bulk insert completado: 
 'inventory_us_2021' (4999 filas)
Ahí va :D
